# This is an example notebook on how to use NAM for shape matching
In this notebook we consider different refinement with the spectral embeddings


In [1]:
from geomfum.shape.mesh import TriangleMesh
from geomfum.refine import ZoomOut

import torch
import numpy as np

import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from model.neural_adjoint_map import NeuralAdjointMap
from sklearn.neighbors import NearestNeighbors

from model.neural_zoomout import NeuralZoomOut


In [2]:
import os
from urllib.request import urlretrieve

faust_url = "https://raw.githubusercontent.com/JM-data/PyFuncMap/4bde4484c3e93bff925a6a82da29fa79d6862f4b/FAUST_shapes_off/"
shape_files = ["tr_reg_080.off", "tr_reg_093.off"]
for fname in shape_files:
    url = faust_url + fname
    out_path = os.path.join("../data/", fname)
    urlretrieve(url, out_path)


In [3]:
mesh1 = TriangleMesh.from_file("../data/tr_reg_080.off")
mesh2 = TriangleMesh.from_file("../data/tr_reg_093.off")

eigvals1, eigvecs1 = mesh1.laplacian.find_spectrum(spectrum_size=200)
eigvals2, eigvecs2 = mesh2.laplacian.find_spectrum(spectrum_size=200)

c:\Users\giuli\OneDrive\Research\NAM-NeuralAdjointMaps\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
p2p_gt = np.arange(mesh1.n_vertices)

# Initial Map

In [5]:
k = 20
C21 = np.linalg.pinv(eigvecs1[:, :k]) @ eigvecs2[p2p_gt, :20]

A12 = C21.T
# Fit KNN on eigvecs2
knn = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(eigvecs1[:, :20])
distances, indices = knn.kneighbors(eigvecs2[:, :k] @ A12)

p2p_ini = indices.flatten()

print("Conversion Error:", np.mean((mesh2.vertices[p2p_ini] - mesh2.vertices) ** 2))

Conversion Error: 0.001446526388343697


# ZoomOut

In [6]:
zoomout = ZoomOut(nit=18, step=10)
ref_fmap = zoomout(C21, mesh1.basis, mesh2.basis)

In [7]:
# Fit KNN on eigvecs2
knn = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(eigvecs1)
distances, indices = knn.kneighbors(eigvecs2 @ ref_fmap)

p2p = indices.flatten()

print("Conversion Error:", np.mean((mesh2.vertices[p2p] - mesh2.vertices) ** 2))

Conversion Error: 0.002439187617084261


# NeuralZoomOut

In [ ]:
nam = NeuralAdjointMap(
    torch.tensor(eigvecs1[:, :20]).to(torch.float32).cuda(),
    torch.tensor(eigvecs2[:, :20]).to(torch.float32).cuda(),
)

nam.optimize_from_p2p(p2p_ini)


RuntimeError: Cannot access accelerator device when none is available.

In [ ]:
nzo = NeuralZoomOut(nit=19, step=10)
nam = nzo(
    nam,
    torch.tensor(eigvecs1).to(torch.float32).cuda(),
    torch.tensor(eigvecs2).to(torch.float32).cuda(),
)

emb2_nn = nam(
    torch.tensor(eigvecs2).to(torch.float32).cuda(),
)


In [ ]:
knn = NearestNeighbors(n_neighbors=1, algorithm="auto").fit(eigvecs1)
distances, indices = knn.kneighbors(emb2_nn.detach().cpu().numpy())

p2p = indices.flatten()

In [ ]:
print("Conversion Error:", np.mean((mesh2.vertices[p2p] - mesh2.vertices) ** 2))

In [ ]:
nam